In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.2
    resonator_length_2.append(a)
print(resonator_length_2)


[7.253254683870284, 7.024788339841421, 6.80834654020913]


# Flipchip

In [6]:
#V10부터 pad gap 바뀜
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            pad_gap='20um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="110um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [73]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [74]:
hfss.start()

INFO 10:18AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:18AM [load_ansys_project]: 	Opened Ansys App
INFO 10:18AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:18AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 10:18AM [connect_design]: No active design found (or error getting active design).
INFO 10:18AM [connect]: 	 Connected to project "Project20". No design detected


True

In [76]:
hfss.activate_ansys_design("flip_V10-5.1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

11:02AM 15s WARNING [activate_ansys_design]: The design_name=flip_V10-5.1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:02AM [connect_design]: 	Opened active design
	Design:    flip_V10-5.1 [Solution type: Eigenmode]
WARNING 11:02AM [connect_setup]: 	No design setup detected.
WARNING 11:02AM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:02AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [77]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.59',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.59',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [15]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 03:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpvwdk2wxs.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5590000000, Maxwell, 1, False
INFO 03:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmphmmdpxzk.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5590000000, Maxwell, 1, False
INFO 03:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp6g3jth5i.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5590000000, Maxwell, 2, False
INFO 03:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpktwgsly1.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5590000000, Maxwell, 3, False
INFO 03:08PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3fslt73r.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5590000000, Maxwell, 4, False
INFO 03:08

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,384.370,-72.60900,-53.008,-199.59000
pad_bot_Q0,-72.609,111.87000,-37.464,-0.67174
pad_top_Q0,-53.008,-37.46400,111.680,-20.22500
quad_coupler_c0,-199.590,-0.67174,-20.225,221.66000


In [16]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.84
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.233069 [GHz]
EC 254.561825 [MHz]
EJ 14.854145 [GHz]
alpha -288.672023 [MHz]
dispersion 2.471637 [KHz]
Lq 10.995564 [nH]
Cq 76.092431 [fF]
T1 47.422175 [us]

**Coupling Properties**

tCqbus1 -9.780034 [fF]
gbus1_in_MHz -129.843345 [MHz]
χ_bus1 -3.263846 [MHz]
1/T1bus1 3356.129125 [Hz]
T1bus1 47.422175 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.633487,297.641012,14.854145,-341.910221,11.89982,[-11.99231506006353],[-0.05329121211487465],0.053291,11.992315
2,5.541732,287.432884,14.854145,-329.187165,8.47193,[-201.57144514880298],[-12.839941236934386],12.839941,201.571445
3,5.443248,276.700153,14.854145,-315.882493,5.807345,[-186.1533715206259],[-9.30087449067141],9.300874,186.153372
4,5.43647,275.970014,14.854145,-314.98007,5.655472,[-174.08840262537282],[-8.045529766106888],8.045530,174.088403
5,5.38512,270.473432,14.854145,-308.197476,4.616536,[-151.56548104052013],[-5.617474176356951],5.617474,151.565481
6,5.361136,267.92752,14.854145,-305.062461,4.193297,[-142.59037104179802],[-4.7875896263776845],4.787590,142.590371
7,5.301382,261.643401,14.854145,-297.341988,3.287321,[-135.2690345714959],[-3.9276904344630155],3.927690,135.269035
8,5.278689,259.278862,14.854145,-294.443524,2.99267,[-132.8101811401616],[-3.6574510205892437],3.657451,132.810181
9,5.246662,255.96224,14.854145,-290.384009,2.617526,[-131.25564950010337],[-3.4037797802348093],3.403780,131.255650
10,5.233069,254.561825,14.854145,-288.672023,2.471637,[-129.84334548394565],[-3.2638462223070848],3.263846,129.843345


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
